In [1]:
from typing import TypedDict, List, Any
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import os

In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [ ]:
#DOMAIN = os.getenv('DOMAIN')
#ADMIN_EMAIL = os.getenv('ADMIN_EMAIL')

In [9]:
class AgentState(TypedDict):
    messages: List[HumanMessage]

In [10]:
llm = ChatOpenAI(model="gpt-4o")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable